In [1]:
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21.0.1+12-LTS-29, mixed mode, sharing)
  Starting server from C:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\38066\AppData\Local\Temp\tmpldcitrph
  JVM stdout: C:\Users\38066\AppData\Local\Temp\tmpldcitrph\h2o_38066_started_from_python.out
  JVM stderr: C:\Users\38066\AppData\Local\Temp\tmpldcitrph\h2o_38066_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Kiev
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,3 months and 26 days
H2O_cluster_name:,H2O_from_python_38066_euy3wc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.952 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


logP STANDART scaler

In [2]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\train_logp_standart_scaler.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\test_logp_standart_scaler.csv', index_col=0)

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
x = train.columns
x.remove('logP')
x.remove('fold_id')
y = 'logP'

In [4]:
from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_features", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\train_logp_minmax.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\test_logp_minmax.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=10, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_train_cv, metrics_oos_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_oos_{cross_val_index}_mse"] = metrics_oos_cv.mse()
            cv_dict[f"cv_oos_{cross_val_index}_mae"] = metrics_oos_cv.mae()
            cv_dict[f"cv_oos_{cross_val_index}_r2"] = metrics_oos_cv.r2()
            cv_dict[f"cv_train_{cross_val_index}_mse"] = metrics_train_cv.mse()
            cv_dict[f"cv_train_{cross_val_index}_mae"] = metrics_train_cv.mae()
            cv_dict[f"cv_train_{cross_val_index}_r2"] = metrics_train_cv.r2()
            total_r_2_score += metrics_oos_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('logP')
        x.remove('fold_id')
        y = 'logP'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules and linear"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance_train_cv = rfit.model_performance(test_data=train)
        performance_oos_cv = rfit.model_performance(test_data=test)
        return performance_train_cv, performance_oos_cv


Trainer()

NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
████████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█████████████████████

In [5]:
crossvals_list

[{'params': {'max_num_rules': 762, 'max_rule_length': 12, 'min_num_rules': 3},
  'cv_oos_0_mse': 0.03827479534357713,
  'cv_oos_0_mae': 0.15630166805997645,
  'cv_oos_0_r2': 0.8721823506654762,
  'cv_train_0_mse': 0.060284939903568556,
  'cv_train_0_mae': 0.18609758547834954,
  'cv_train_0_r2': 0.8205717672854382,
  'cv_oos_1_mse': 0.08302379609946488,
  'cv_oos_1_mae': 0.21797027421923937,
  'cv_oos_1_r2': 0.7528932925667676,
  'cv_train_1_mse': 0.03419810187332645,
  'cv_train_1_mae': 0.15267390053630972,
  'cv_train_1_r2': 0.8857963588331851},
 {'params': {'max_num_rules': 537, 'max_rule_length': 11, 'min_num_rules': 4},
  'cv_oos_0_mse': 0.03827476196534644,
  'cv_oos_0_mae': 0.15630151611511922,
  'cv_oos_0_r2': 0.8721824621311797,
  'cv_train_0_mse': 0.06028496554363322,
  'cv_train_0_mae': 0.18609764816975882,
  'cv_train_0_r2': 0.8205716909719925,
  'cv_oos_1_mse': 0.0830241383739798,
  'cv_oos_1_mae': 0.2179732754200187,
  'cv_oos_1_r2': 0.7528922738428306,
  'cv_train_1_mse':

In [8]:
from h2o.estimators import H2ORuleFitEstimator

rfit = H2ORuleFitEstimator(
                           max_rule_length=16,
                           max_num_rules=982,
                           min_rule_length=8,
                           seed=1,
                           model_type="rules and linear")
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.03304 ),37210,11,1,37185.0,450.0,450.0,0.0,16.0,10.677778,0.0,99.0,82.63333


In [23]:
print(rfit.rule_importance())

Rule Importance: 
    variable               coefficient    support    rule
--  ---------------------  -------------  ---------  ------
    linear.identificator   -0.200378      1
    linear.nC              0.150803       1
    linear.TASA            0.125385       1
    linear.nO              -0.0991345     1
    linear.FPSA3           -0.0655568     1
    linear.nHRing          -0.0585389     1
    linear.RPCS            -0.0379213     1
    linear.WPSA5           0.0322494      1
    linear.GeomShapeIndex  0.0317859      1
    linear.mol_volume      0.0251743      1
    linear.f_to_fg         -0.0242538     1


In [6]:
performance = rfit.model_performance(test_data=test)

print(performance)

ModelMetricsRegressionGLM: rulefit
** Reported on test data. **

MSE: 0.02866347750725269
RMSE: 0.1693029164168553
MAE: 0.13634058649133893
RMSLE: 0.055419062725898506
Mean Residual Deviance: 0.02866347750725269
R^2: 0.8923470810243922
Null degrees of freedom: 24
Residual degrees of freedom: 13
Null deviance: 6.687705755427244
Residual deviance: 0.7165869376813173
AIC: 8.14363847692238


In [28]:
performance.r2()

0.8956729514470931

In [7]:
performance.mse()

0.02866347750725269

In [8]:
performance.mae()

0.13634058649133893

logP STANDART scaler

In [11]:
crossvals_list

[{'params': {'max_num_rules': 417, 'max_rule_length': 13, 'min_num_rules': 3},
  'cv_0_mse': 0.03827472398886842,
  'cv_0_mae': 0.15630146526150746,
  'cv_0_r2': 0.8721825889526063,
  'cv_1_mse': 0.08302338406862465,
  'cv_1_mae': 0.21797112626417497,
  'cv_1_r2': 0.752894518908962},
 {'params': {'max_num_rules': 655, 'max_rule_length': 18, 'min_num_rules': 8},
  'cv_0_mse': 0.03827485177769583,
  'cv_0_mae': 0.1563018574639205,
  'cv_0_r2': 0.8721821622052551,
  'cv_1_mse': 0.08301941015937186,
  'cv_1_mae': 0.2179669642535507,
  'cv_1_r2': 0.7529063465978563},
 {'params': {'max_num_rules': 92, 'max_rule_length': 12, 'min_num_rules': 5},
  'cv_0_mse': 0.03813782730470032,
  'cv_0_mae': 0.15666596422027557,
  'cv_0_r2': 0.8726397517464236,
  'cv_1_mse': 0.08711204660770157,
  'cv_1_mae': 0.22446078101347083,
  'cv_1_r2': 0.7407252856854354},
 {'params': {'max_num_rules': 568, 'max_rule_length': 14, 'min_num_rules': 5},
  'cv_0_mse': 0.03827479735102099,
  'cv_0_mae': 0.1563016769972262

In [12]:
from h2o.estimators import H2ORuleFitEstimator

rfit = H2ORuleFitEstimator(
                           max_rule_length=10,
                           max_num_rules=693,
                           min_rule_length=6,
                           seed=1,
                           model_type="rules and linear")
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.03304 ),16511,11,1,16486.0,250.0,250.0,0.0,10.0,7.996,0.0,98.0,65.944


PKA STANDART SCALER

In [4]:
import pandas as pd

from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_features", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\train_pka_standart_scaler.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\test_pka_standart_scaler.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=10, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_train_cv, metrics_oos_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_oos_{cross_val_index}_mse"] = metrics_oos_cv.mse()
            cv_dict[f"cv_oos_{cross_val_index}_mae"] = metrics_oos_cv.mae()
            cv_dict[f"cv_oos_{cross_val_index}_r2"] = metrics_oos_cv.r2()
            cv_dict[f"cv_train_{cross_val_index}_mse"] = metrics_train_cv.mse()
            cv_dict[f"cv_train_{cross_val_index}_mae"] = metrics_train_cv.mae()
            cv_dict[f"cv_train_{cross_val_index}_r2"] = metrics_train_cv.r2()
            total_r_2_score += metrics_oos_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('pKa')
        x.remove('fold_id')
        y = 'pKa'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules and linear"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance_train_cv = rfit.model_performance(test_data=train)
        performance_oos_cv = rfit.model_performance(test_data=test)
        return performance_train_cv, performance_oos_cv


Trainer()

NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
████████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                    

In [5]:
crossvals_list

[{'params': {'max_num_rules': 172, 'max_rule_length': 15, 'min_num_rules': 6},
  'cv_oos_0_mse': 0.9471910060327727,
  'cv_oos_0_mae': 0.8024925494524501,
  'cv_oos_0_r2': 0.8525245758846333,
  'cv_train_0_mse': 0.8036240381144619,
  'cv_train_0_mae': 0.7338148576018086,
  'cv_train_0_r2': 0.8773218095871371,
  'cv_oos_1_mse': 0.7294517012233461,
  'cv_oos_1_mae': 0.7098091363109247,
  'cv_oos_1_r2': 0.8886446765459766,
  'cv_train_1_mse': 0.815093961283493,
  'cv_train_1_mae': 0.7406416673302215,
  'cv_train_1_r2': 0.873091776770948},
 {'params': {'max_num_rules': 859, 'max_rule_length': 10, 'min_num_rules': 3},
  'cv_oos_0_mse': 0.9471943284924027,
  'cv_oos_0_mae': 0.8024906062534515,
  'cv_oos_0_r2': 0.8525240585854402,
  'cv_train_0_mse': 0.8036211656095975,
  'cv_train_0_mae': 0.7338083890437008,
  'cv_train_0_r2': 0.8773222480928089,
  'cv_oos_1_mse': 0.7294316784604981,
  'cv_oos_1_mae': 0.7097982699765262,
  'cv_oos_1_r2': 0.8886477331448297,
  'cv_train_1_mse': 0.815058904739

In [5]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\train_pka_standart_scaler.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\test_pka_standart_scaler.csv', index_col=0)

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

x = train.columns
x.remove('pKa')
x.remove('fold_id')
y = 'pKa'

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
from h2o.estimators import H2ORuleFitEstimator

{'max_features': 9, 'max_rule_length': 10, 'min_num_rules': 3}

rfit = H2ORuleFitEstimator(
    max_rule_length=10,
    max_num_rules=9,
    min_rule_length=3,
    seed=1,
    model_type="rules and linear"
)
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.2323 ),18441,3,1,18417.0,400.0,400.0,0.0,10.0,6.5,0.0,96.0,46.0425


PKA MINMAX SCALER

In [7]:
from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_features", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\train_pka_minmax.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\test_pka_minmax.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=10, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_{cross_val_index}_mse"] = metrics_cv.mse()
            cv_dict[f"cv_{cross_val_index}_mae"] = metrics_cv.mae()
            cv_dict[f"cv_{cross_val_index}_r2"] = metrics_cv.r2()
            total_r_2_score += metrics_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('pKa')
        x.remove('fold_id')
        y = 'pKa'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules and linear"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance = rfit.model_performance(test_data=test)
        r_2_score = performance.r2()
        return performance


Trainer()

NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
████████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
████

In [9]:
crossvals_list

[{'params': {'max_num_rules': 798, 'max_rule_length': 15, 'min_num_rules': 7},
  'cv_0_mse': 0.9471949924513353,
  'cv_0_mae': 0.8024905013255788,
  'cv_0_r2': 0.8525239552085875,
  'cv_1_mse': 0.7294320104395806,
  'cv_1_mae': 0.7097974916496892,
  'cv_1_r2': 0.8886476824661651},
 {'params': {'max_num_rules': 240, 'max_rule_length': 18, 'min_num_rules': 8},
  'cv_0_mse': 0.9471948448643953,
  'cv_0_mae': 0.8024906103536471,
  'cv_0_r2': 0.8525239781875288,
  'cv_1_mse': 0.7294252250734733,
  'cv_1_mae': 0.7097934506593858,
  'cv_1_r2': 0.8886487182943581},
 {'params': {'max_num_rules': 294, 'max_rule_length': 11, 'min_num_rules': 4},
  'cv_0_mse': 0.9471912238550948,
  'cv_0_mae': 0.8024924972013505,
  'cv_0_r2': 0.8525245419702072,
  'cv_1_mse': 0.7294296617038188,
  'cv_1_mae': 0.709797862268682,
  'cv_1_r2': 0.888648041015237},
 {'params': {'max_num_rules': 427, 'max_rule_length': 19, 'min_num_rules': 6},
  'cv_0_mse': 0.9471910013290176,
  'cv_0_mae': 0.8024925654084023,
  'cv_0_r

In [8]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\train_pka_minmax.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\pKa\test_pka_minmax.csv', index_col=0)

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

x = train.columns
x.remove('pKa')
x.remove('fold_id')
y = 'pKa'

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
from h2o.estimators import H2ORuleFitEstimator

{'max_features': 240, 'max_rule_length': 18, 'min_num_rules': 8}

rfit = H2ORuleFitEstimator(
    max_rule_length=18,
    max_num_rules=240,
    min_rule_length=8,
    seed=1,
    model_type="rules and linear"
)
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.2323 ),46127,3,1,46103.0,550.0,550.0,0.0,16.0,10.981818,0.0,101.0,83.82364
